In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
from pathlib import Path
import pandas as pd
import numpy as np
from spotifuncs import *

In [2]:
path = Path("C:/Users/ms101/OneDrive/DataScience_ML/projects/spotify_app")

In [3]:
with open(path / "client_s.txt") as f:
    content = f.readlines()
content = [x.strip() for x in content]

client_id = content[0]
client_secret = content[1]

In [4]:
with open(path / "usernames.txt") as f:
    usernames = f.readlines()
usernames = [x.strip() for x in usernames]

username1 = usernames[0]
username2 = usernames[1]

In [5]:
scope = "user-library-read user-read-recently-played user-top-read playlist-modify-public playlist-read-private playlist-read-collaborative"

redirect_uri = "https://developer.spotify.com/dashboard/applications/4a4e029d299a4241873db8300038bf0a"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, 
                                                      client_secret=client_secret)


In [6]:
sp_m = authenticate(redirect_uri, client_credentials_manager, username1, scope, client_id, client_secret)
sp_t = authenticate(redirect_uri, client_credentials_manager, username2, scope, client_id, client_secret)

In [7]:
def get_dfs(sp):
    ##queries
    #user top tracks
    top_tracks_short = sp.current_user_top_tracks(limit = 50,offset=0,time_range='short_term')
    top_tracks_med = sp.current_user_top_tracks(limit = 50,offset=0,time_range='medium_term')
    top_tracks_long = sp.current_user_top_tracks(limit = 50,offset=0,time_range='long_term')
    
    #combine the top_tracks
    top_tracks_short_df = append_audio_features(create_df_top_songs(top_tracks_short),sp)
    top_tracks_med_df = append_audio_features(create_df_top_songs(top_tracks_med),sp)
    top_tracks_long_df = append_audio_features(create_df_top_songs(top_tracks_long),sp)
    top_tracks_df = pd.concat([top_tracks_short_df,top_tracks_med_df,top_tracks_long_df]).drop_duplicates().reset_index(drop = True)
        
    #user top artists
    top_artists_long = sp.current_user_top_artists(limit = 50, time_range = "long_term")
    top_artists_med = sp.current_user_top_artists(limit = 50, time_range = "medium_term")
    top_artists_short = sp.current_user_top_artists(limit = 50, time_range = "short_term")
    
    artists_short_df = top_artists_from_API(top_artists_short)
    artists_med_df = top_artists_from_API(top_artists_med)
    artists_long_df = top_artists_from_API(top_artists_long)
    artists_df = pd.concat([artists_short_df,artists_med_df,artists_long_df])
    artists_df["genres"] = artists_df["genres"].apply(lambda x: ",".join(x))
    artists_df.drop_duplicates().reset_index(drop = True)
    
    #user saved tracks
    user_saved_tracks = sp.current_user_saved_tracks(limit = 50)
    saved_tracks_df = create_df_saved_songs(user_saved_tracks)
    
        
    return top_tracks_df,artists_df,saved_tracks_df

In [8]:
top_tracks_m, artists_m, saved_tracks_m = get_dfs(sp_m)

In [9]:
top_tracks_t, artists_t, saved_tracks_t = get_dfs(sp_t)

In [10]:
artists_t

,name,id,genres,popularity,uri
0,Emancipator,6HCnsY0Rxi3cg53xreoAIm,"downtempo,electronica,livetronica,nu jazz,trip...",61,spotify:artist:6HCnsY0Rxi3cg53xreoAIm
1,Arctic Monkeys,7Ln80lUS6He07XvHI8qqHH,"garage rock,modern rock,permanent wave,rock,sh...",86,spotify:artist:7Ln80lUS6He07XvHI8qqHH
2,Masego,3ycxRkcZ67ALN3GQJ57Vig,"alternative r&b,indie soul",74,spotify:artist:3ycxRkcZ67ALN3GQJ57Vig
3,Element Of Crime,3FweAJRBCbUOGR6jULfaRi,"german indie,german pop,german rock,liedermacher",49,spotify:artist:3FweAJRBCbUOGR6jULfaRi
4,Mac Miller,4LLpKhyESsyAXpc4laK94U,"hip hop,pittsburgh rap,pop rap,rap",85,spotify:artist:4LLpKhyESsyAXpc4laK94U
...,...,...,...,...,...
45,257ers,6ihLfpY3cmdGyWEnItn30w,"antideutsche,deep german hip hop,german hip ho...",62,spotify:artist:6ihLfpY3cmdGyWEnItn30w
46,Eminem,7dGJo4pcD2V6oG8kP0tJRR,"detroit hip hop,hip hop,rap",92,spotify:artist:7dGJo4pcD2V6oG8kP0tJRR
47,MEUTE,1z5xbcOeFRQXBVDpvRPh8H,"german dance,hamburg electronic,livetronica",53,spotify:artist:1z5xbcOeFRQXBVDpvRPh8H
48,Max Herre,7IpWQKu80qQvyer3LO6SW3,"german alternative rap,german hip hop,german pop",53,spotify:artist:7IpWQKu80qQvyer3LO6SW3


In [11]:
artists_m

,name,id,genres,popularity,uri
0,Joyner Lucas,6C1ohJrd5VydigQtaGy5Wa,"boston hip hop,pop rap,rap",78,spotify:artist:6C1ohJrd5VydigQtaGy5Wa
1,Fynn Kliemann,2YIxQoGvBukvoC1CbJ7opS,german pop,65,spotify:artist:2YIxQoGvBukvoC1CbJ7opS
2,Geegun,5W7N6u4EjCEMKj7bDyzPEC,"russian dance,russian dance pop,russian hip ho...",57,spotify:artist:5W7N6u4EjCEMKj7bDyzPEC
3,Gidge,7aibxFH8hIlSUlXgshCgAP,"electronica,italian electronica,microhouse",49,spotify:artist:7aibxFH8hIlSUlXgshCgAP
4,Peter Fox,6rqlONGmPuP2wJVSfliLBI,"german hip hop,german pop,german rock",61,spotify:artist:6rqlONGmPuP2wJVSfliLBI
...,...,...,...,...,...
45,Chris Rock,36eSjIksD6fehqxyDUHDA3,"black comedy,comedy",48,spotify:artist:36eSjIksD6fehqxyDUHDA3
46,Carnage,7CCjtD0hCK005Bvg2WG1a7,"edm,electro house,electronic trap,pop rap,rap,...",64,spotify:artist:7CCjtD0hCK005Bvg2WG1a7
47,Wardruna,0NJ6wlOAsAJ1PN4VRdTPKA,"medieval folk,nordic folk,rune folk,viking folk",62,spotify:artist:0NJ6wlOAsAJ1PN4VRdTPKA
48,Motörhead,1DFr97A9HnbV3SKTJFu62M,"album rock,alternative metal,hard rock,metal,r...",68,spotify:artist:1DFr97A9HnbV3SKTJFu62M


In [12]:
def dataframe_difference(df1, df2, which=None):
    """Find rows which are different between two DataFrames."""
    comparison_df = df1.merge(
        df2,
        indicator=True,
        how='outer'
    )
    if which is None:
        diff_df = comparison_df[comparison_df['_merge'] != 'both']
    else:
        diff_df = comparison_df[comparison_df['_merge'] == which]
    diff_df.drop("_merge",axis = 1, inplace = True)
    return diff_df.drop_duplicates().reset_index(drop = True)

In [13]:
common_artists = dataframe_difference(artists_m,artists_t, which = "both")
common_artists

C:\Users\ms101\Anaconda3\envs\spotify_app\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,name,id,genres,popularity,uri
0,Joyner Lucas,6C1ohJrd5VydigQtaGy5Wa,"boston hip hop,pop rap,rap",78,spotify:artist:6C1ohJrd5VydigQtaGy5Wa
1,Mac Miller,4LLpKhyESsyAXpc4laK94U,"hip hop,pittsburgh rap,pop rap,rap",85,spotify:artist:4LLpKhyESsyAXpc4laK94U
2,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,"canadian contemporary r&b,canadian pop,pop",94,spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ
3,KitschKrieg,5tHiL8SKSaZGMBUPIiSmX4,"german hip hop,hamburg hip hop",70,spotify:artist:5tHiL8SKSaZGMBUPIiSmX4
4,Monolink,2I4hRNCYkPKJQlkoEZKjYx,"electronica,organic house,tech house",62,spotify:artist:2I4hRNCYkPKJQlkoEZKjYx
5,ELIF,65AzRSW0jKSs0WtttEXrOw,"frauenrap,german pop,german singer-songwriter",67,spotify:artist:65AzRSW0jKSs0WtttEXrOw
6,Seeed,5ISjkNS17JpCwiFtW80lpV,"german hip hop,german pop,german reggae,german...",66,spotify:artist:5ISjkNS17JpCwiFtW80lpV
7,Egor Kreed,2KoLmBXwsgMkfAvoPBlPmb,"russian hip hop,russian pop",65,spotify:artist:2KoLmBXwsgMkfAvoPBlPmb
8,Modeselektor,2jYMYP2SVifgmzNRQJx3SJ,"electronica,microhouse,minimal techno",50,spotify:artist:2jYMYP2SVifgmzNRQJx3SJ
9,Christian Löffler,3tSvlEzeDnVbQJBTkIA6nO,"electronica,hamburg electronic,microhouse,mini...",62,spotify:artist:3tSvlEzeDnVbQJBTkIA6nO


In [14]:
dataframe_difference(top_tracks_m,top_tracks_t,which = "both")

C:\Users\ms101\Anaconda3\envs\spotify_app\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Who - Single Version,Who,0CjBORMsmiQNe3vPDcNIvk,Modeselektor,207988,43,0.76,0.910,7,-8.472,1,0.0484,0.0442,0.006620,0.108,0.392,142.982
1,Bläulich,Treppenhaus,2WRTnY0slmFgWcrmEr8dPj,Apache 207,196213,70,0.79,0.704,10,-7.935,0,0.4170,0.0690,0.000658,0.113,0.212,154.007


In [15]:
dataframe_difference(saved_tracks_m,saved_tracks_t, which = "both")

C:\Users\ms101\Anaconda3\envs\spotify_app\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,track_name,album,track_id,artist,duration,popularity


In [16]:
new_playlist_df = dataframe_difference(top_tracks_m,top_tracks_t,which = "both")

C:\Users\ms101\Anaconda3\envs\spotify_app\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [17]:
filtered_top_m = top_tracks_m[top_tracks_m["artist"].isin(common_artists["name"])]
filtered_top_m

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
1,Evolution,Evolution,2VopDw2GlF3uwD1kihHmTT,Joyner Lucas,153250,62,0.687,0.819,9,-6.670,0,0.4310,0.21800,0.000000,0.3920,0.5680,81.185
3,Fall Slowly (feat. Ashanti) - Extended Version,Evolution,7wK4pOTZKVaAJ00rziu901,Joyner Lucas,292056,58,0.542,0.453,6,-11.208,0,0.2400,0.20400,0.000007,0.1370,0.0375,88.310
7,Oh Junge,KitschKrieg,0JqbpesudPE6j901fBEzo2,KitschKrieg,193946,58,0.878,0.515,1,-7.715,1,0.1880,0.02920,0.000000,0.0835,0.2690,156.000
10,Who - Single Version,Who,0CjBORMsmiQNe3vPDcNIvk,Modeselektor,207988,43,0.760,0.910,7,-8.472,1,0.0484,0.04420,0.006620,0.1080,0.3920,142.982
12,Zim Zimma,Evolution,17nPeSliosCi427f0lUb75,Joyner Lucas,239702,70,0.883,0.621,11,-6.063,0,0.2120,0.08710,0.000000,0.4990,0.6760,149.052
22,ADHD,ADHD,4X4v3KtkUXwXvDBw5KS9cp,Joyner Lucas,205872,71,0.563,0.780,10,-6.663,1,0.0782,0.00525,0.000008,0.4180,0.3170,83.913
24,Lambo Lambo,KitschKrieg,7oqvRZNv4dUV8CgQWtIAMe,KitschKrieg,214991,59,0.876,0.400,5,-9.748,0,0.1290,0.49400,0.000009,0.1000,0.4840,144.938
33,Devil's Work,ADHD,3FpmWMmuQfQLpyVD7t5CCE,Joyner Lucas,285048,59,0.575,0.746,2,-5.204,1,0.3240,0.14200,0.000000,0.1060,0.2350,96.113
52,ALLES HELAL,NACHT,2KAbQ3PsETrr86R39pru7k,ELIF,175062,59,0.727,0.600,4,-6.186,0,0.0376,0.16400,0.000017,0.0793,0.1440,92.024
54,KANN DAS BITTE SO BLEIBEN,NACHT,45HOck8XCgrSlVUQHHOHMz,ELIF,164511,49,0.705,0.656,6,-6.407,0,0.0468,0.18000,0.005810,0.1890,0.4700,150.066


In [18]:
filtered_top_t = top_tracks_t[top_tracks_t["artist"].isin(common_artists["name"])]
filtered_top_t

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
2,505,Favourite Worst Nightmare,0BxE4FqsDD1Ot4YuBXwAPp,Arctic Monkeys,253586,80,0.526,0.866,0,-5.822,1,0.0568,0.00287,0.000078,0.0945,0.248,140.266
13,Не могу,Что они знают?,72aW82jpOZ6L0Ran6cZK2q,Egor Kreed,198037,46,0.817,0.527,6,-9.389,0,0.0461,0.28200,0.000006,0.3700,0.360,109.994
15,Why'd You Only Call Me When You're High?,AM,086myS9r57YsLbJpU0TgK9,Arctic Monkeys,161123,84,0.691,0.631,2,-6.478,1,0.0368,0.04830,0.000011,0.1040,0.800,92.004
19,Do I Wanna Know?,AM,5FVd6KXrgO9B3JPmC8OPst,Arctic Monkeys,272394,85,0.548,0.532,5,-7.596,1,0.0323,0.18600,0.000263,0.2170,0.405,85.030
34,Self Care,Swimming,5bJ1DrEM4hNCafcDd1oxHx,Mac Miller,345133,79,0.520,0.538,1,-8.109,1,0.2060,0.36700,0.001870,0.1190,0.177,141.894
35,Coco L'Eau,Coco L'Eau,5weiiB92gNV7QHFYQXqxZ8,Egor Kreed,130050,64,0.837,0.663,3,-5.071,0,0.0497,0.09490,0.009540,0.1750,0.466,100.006
50,Wealth,Who Else,5aOlYhQsp75cgPov4yjWIe,Modeselektor,247218,42,0.765,0.452,1,-12.346,1,0.0856,0.02040,0.000057,0.0727,0.331,137.972
89,Who - Single Version,Who,0CjBORMsmiQNe3vPDcNIvk,Modeselektor,207988,43,0.760,0.910,7,-8.472,1,0.0484,0.04420,0.006620,0.1080,0.392,142.982
94,Had Enough,Everything's Strange Here,26Td0OU1dTm3Je3ynSLC1a,G-Eazy,193441,58,0.638,0.683,3,-8.121,1,0.0554,0.00512,0.000010,0.0850,0.365,103.023
101,What A Day,BAM BAM,3Zexl3b8tdk5csqUHgTPnZ,Seeed,153759,38,0.435,0.390,9,-8.826,0,0.0271,0.46900,0.000000,0.1730,0.200,132.331


In [19]:
#to not have 2+ songs by the same artist we will sample from the above dataframes
#I will assign weights to the rows depending on how often an artist occurs
from collections import Counter

Counter(filtered_top_m["artist"]) ,  Counter(filtered_top_t["artist"])

(Counter({'Joyner Lucas': 5,
          'KitschKrieg': 2,
          'Modeselektor': 1,
          'ELIF': 15,
          'G-Eazy': 2}),
 Counter({'Arctic Monkeys': 3,
          'Egor Kreed': 2,
          'Mac Miller': 1,
          'Modeselektor': 2,
          'G-Eazy': 1,
          'Seeed': 5,
          'Monolink': 1}))

In [20]:
weights_m = [1/len(filtered_top_m)/7 if Counter(filtered_top_m["artist"])[x] > 2 else 1/len(filtered_top_m) for x in filtered_top_m["artist"]] 

In [21]:
weights_t = [1/len(filtered_top_t)/7 if Counter(filtered_top_t["artist"])[x] > 2 else 1/len(filtered_top_m) for x in filtered_top_t["artist"]] 

I tried the sampling with and without weights 10 times. With weights artists that occur very often in the filtered dataframe are not overrepresented just as planned. without weights the sample sometimes contained just one or two artists which is not desired

In [22]:
sample_n = (25-len(new_playlist_df))//2
if sample_n > 7: sample_n = 7
sample_n

7

In [23]:
filtered_top_m.sample(sample_n,weights = weights_m)

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
86,DU TUST MIR NICHT GUT,NACHT,2pmqIWXxhdvjJfRiwySIZl,ELIF,207279,52,0.860,0.434,4,-7.951,0,0.0677,0.6300,0.000000,0.0752,0.500,133.973
24,Lambo Lambo,KitschKrieg,7oqvRZNv4dUV8CgQWtIAMe,KitschKrieg,214991,59,0.876,0.400,5,-9.748,0,0.1290,0.4940,0.000009,0.1000,0.484,144.938
7,Oh Junge,KitschKrieg,0JqbpesudPE6j901fBEzo2,KitschKrieg,193946,58,0.878,0.515,1,-7.715,1,0.1880,0.0292,0.000000,0.0835,0.269,156.000
139,The Beautiful & Damned,The Beautiful & Damned,2WWruw7ul9N7eqoHELyMc2,G-Eazy,189306,61,0.656,0.804,8,-5.191,0,0.3630,0.1730,0.000000,0.8370,0.314,125.882
33,Devil's Work,ADHD,3FpmWMmuQfQLpyVD7t5CCE,Joyner Lucas,285048,59,0.575,0.746,2,-5.204,1,0.3240,0.1420,0.000000,0.1060,0.235,96.113
70,EIN LETZTES MAL,NACHT,04EJyZSlhPPfFOo1NRn2vl,ELIF,144644,50,0.713,0.549,1,-6.168,1,0.0645,0.0461,0.000041,0.1080,0.393,89.005
10,Who - Single Version,Who,0CjBORMsmiQNe3vPDcNIvk,Modeselektor,207988,43,0.760,0.910,7,-8.472,1,0.0484,0.0442,0.006620,0.1080,0.392,142.982


In [24]:
filtered_top_t.sample(sample_n, weights= weights_t)

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
94,Had Enough,Everything's Strange Here,26Td0OU1dTm3Je3ynSLC1a,G-Eazy,193441,58,0.638,0.683,3,-8.121,1,0.0554,0.00512,0.000010,0.0850,0.365,103.023
89,Who - Single Version,Who,0CjBORMsmiQNe3vPDcNIvk,Modeselektor,207988,43,0.760,0.910,7,-8.472,1,0.0484,0.04420,0.006620,0.1080,0.392,142.982
50,Wealth,Who Else,5aOlYhQsp75cgPov4yjWIe,Modeselektor,247218,42,0.765,0.452,1,-12.346,1,0.0856,0.02040,0.000057,0.0727,0.331,137.972
35,Coco L'Eau,Coco L'Eau,5weiiB92gNV7QHFYQXqxZ8,Egor Kreed,130050,64,0.837,0.663,3,-5.071,0,0.0497,0.09490,0.009540,0.1750,0.466,100.006
34,Self Care,Swimming,5bJ1DrEM4hNCafcDd1oxHx,Mac Miller,345133,79,0.520,0.538,1,-8.109,1,0.2060,0.36700,0.001870,0.1190,0.177,141.894
122,The End,The End,0lwUE9DGjdbbuwbPbxZyzw,Monolink,493599,57,0.967,0.393,11,-13.127,0,0.1970,0.12000,0.302000,0.0646,0.314,114.001
2,505,Favourite Worst Nightmare,0BxE4FqsDD1Ot4YuBXwAPp,Arctic Monkeys,253586,80,0.526,0.866,0,-5.822,1,0.0568,0.00287,0.000078,0.0945,0.248,140.266


In [25]:
new_playlist_df = new_playlist_df.append(filtered_top_m.sample(sample_n,weights = weights_m))
new_playlist_df = new_playlist_df.append(filtered_top_t.sample(sample_n,weights = weights_t))

In [26]:
new_playlist_df = new_playlist_df.drop_duplicates().reset_index(drop=True)
new_playlist_df

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Who - Single Version,Who,0CjBORMsmiQNe3vPDcNIvk,Modeselektor,207988,43,0.760,0.910,7,-8.472,1,0.0484,0.04420,0.006620,0.1080,0.392,142.982
1,Bläulich,Treppenhaus,2WRTnY0slmFgWcrmEr8dPj,Apache 207,196213,70,0.790,0.704,10,-7.935,0,0.4170,0.06900,0.000658,0.1130,0.212,154.007
2,The Beautiful & Damned,The Beautiful & Damned,2WWruw7ul9N7eqoHELyMc2,G-Eazy,189306,61,0.656,0.804,8,-5.191,0,0.3630,0.17300,0.000000,0.8370,0.314,125.882
3,ABER WO BIST DU,NACHT,2dGtLJysTSI9cbQ6TulL8V,ELIF,185274,55,0.780,0.592,9,-6.020,0,0.0631,0.23000,0.000000,0.1840,0.705,148.057
4,FREUNDE,NACHT,0VlFGxW4bynvu8j71kK6ai,ELIF,177740,56,0.786,0.705,3,-5.419,0,0.0852,0.33000,0.000000,0.1260,0.469,82.019
5,I Mean It (feat. Remo),These Things Happen,6jmTHeoWvBaSrwWttr8Xvu,G-Eazy,236480,73,0.712,0.562,10,-6.008,1,0.1290,0.12500,0.000000,0.1360,0.142,140.000
6,ALASKA,NACHT,2YnYp5f38UP6fvf7q2FnPm,ELIF,217766,48,0.743,0.671,1,-5.594,0,0.0589,0.05810,0.000000,0.1950,0.484,76.998
7,FEUER,NACHT,0Se4w42WIJiTgld4SYbv8S,ELIF,203386,45,0.769,0.604,8,-6.769,1,0.0455,0.03900,0.000277,0.0738,0.227,97.996
8,Lambo Lambo,KitschKrieg,7oqvRZNv4dUV8CgQWtIAMe,KitschKrieg,214991,59,0.876,0.400,5,-9.748,0,0.1290,0.49400,0.000009,0.1000,0.484,144.938
9,Wealth,Who Else,5aOlYhQsp75cgPov4yjWIe,Modeselektor,247218,42,0.765,0.452,1,-12.346,1,0.0856,0.02040,0.000057,0.0727,0.331,137.972


In [27]:
#sample the remaining 25-len(new_playlist_df) from saved_tracks
#first get audio_features
saved_tracks_m = append_audio_features(saved_tracks_m, sp_m)
saved_tracks_t = append_audio_features(saved_tracks_t,sp_t)
#put that into function above!!

In [28]:
#filter again so artists are not already in new_playlist_df
filtered_saved_m = saved_tracks_m[~saved_tracks_m["artist"].isin(new_playlist_df["artist"])]
filtered_saved_m

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,I Fell In Love,Autumn Bells,0zrHx4EhLePXUa8KhGpS3E,Gidge,385726,44,0.539,0.610,8,-10.999,0,0.1340,0.121000,0.809000,0.0857,0.03950,123.218
1,The Beginning,The Beginning,4njhWDhTAjhReWtYkiMH9t,NR:TN,201904,44,0.783,0.714,7,-7.408,1,0.2460,0.243000,0.860000,0.4460,0.03410,125.955
2,Between Breaths,Stateless,4DJGTc1OsgqFsUGI6W8Mtx,Riyoon,479885,44,0.738,0.665,7,-10.441,0,0.0438,0.007480,0.877000,0.5820,0.03470,99.992
3,Berlin Nights,Berlin Nights,6gG1R1bFdJeNc2ERAwXxCb,Vnce Dolanbay,292115,39,0.901,0.457,10,-13.238,0,0.1630,0.229000,0.424000,0.0977,0.53100,127.999
4,Fall Slowly (feat. Ashanti) - Extended Version,Evolution,7wK4pOTZKVaAJ00rziu901,Joyner Lucas,292056,58,0.542,0.453,6,-11.208,0,0.2400,0.204000,0.000007,0.1370,0.03750,88.310
5,Evolution,Evolution,2VopDw2GlF3uwD1kihHmTT,Joyner Lucas,153250,62,0.687,0.819,9,-6.670,0,0.4310,0.218000,0.000000,0.3920,0.56800,81.185
6,Zim Zimma,Evolution,17nPeSliosCi427f0lUb75,Joyner Lucas,239702,70,0.883,0.621,11,-6.063,0,0.2120,0.087100,0.000000,0.4990,0.67600,149.052
7,Money In The Grave (Drake ft. Rick Ross),The Best In The World Pack,5ry2OE6R2zPQFDO85XkgRb,Drake,205426,82,0.831,0.502,10,-4.045,0,0.0460,0.101000,0.000000,0.1220,0.10100,100.541
8,Beifahrersitz,Beifahrersitz,01qOl2pM8emx1sxdBQc05g,LEA,199586,71,0.712,0.774,6,-3.967,0,0.1330,0.346000,0.000000,0.1760,0.47100,159.977
9,Twingo,POP,6vFEvkXOnOTUMgPHxogIRK,Fynn Kliemann,186984,57,0.619,0.638,2,-5.754,0,0.0906,0.297000,0.000001,0.4400,0.36000,141.967


In [29]:
filtered_saved_t = saved_tracks_t[~saved_tracks_t["artist"].isin(new_playlist_df["artist"])]
filtered_saved_t

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Take Some Time - Emancipator Remix,Take Some Time (Emancipator Remix),28M2ugvRSIa4MIKmiiwNao,Wilderado,290428,41,0.545,0.596,5,-8.216,1,0.0289,0.02490,0.025200,0.2190,0.1820,95.002
1,Time for Space,Baralku,3dJXvBddoH1AGLpKvmbYDA,Emancipator,433613,54,0.610,0.313,5,-13.138,0,0.0323,0.39000,0.840000,0.0899,0.0870,80.009
2,Shuggie - Bonus Track,Her,24V1nw6aGxnIVuWHEGH4e1,Her,259519,45,0.751,0.319,5,-9.445,0,0.0572,0.63900,0.000246,0.1320,0.1650,134.899
3,Borboletas,Borboletas,5dYAXtlAkKVE2I4PYMyHie,Maple Syrup,145588,45,0.740,0.477,6,-8.230,0,0.0598,0.34800,0.276000,0.0918,0.5270,169.991
4,When The Interlude Sounds Like Falling Rain,What Are You Into EP,5lWVqc9kPplvlDmLtFls02,Nandu,300668,43,0.811,0.650,5,-11.214,0,0.0680,0.36400,0.923000,0.1080,0.7130,106.017
5,Phía Sau Đôi Mắt,Phía Sau Đôi Mắt,3DJZM2sfbUleDaCLRXWSco,Anh bạn Thành,174206,41,0.545,0.398,7,-13.675,0,0.3170,0.90500,0.000242,0.1120,0.1560,145.513
6,Suite No. 2 for Jazz Orchestra: VII. Waltz II,Schostakowitsch: Jazz-Suiten,416revLjetHx8BziPfxhRn,Dmitri Shostakovich,231640,13,0.194,0.187,0,-15.575,0,0.0306,0.96000,0.916000,0.0717,0.3640,177.339
7,Carmen,Born To Die,30yDbkiLvSiYx3pox7odMR,Lana Del Rey,248720,39,0.295,0.424,1,-9.562,0,0.0431,0.08200,0.000303,0.1290,0.0378,200.272
8,Zukunft,12,2yI3NNFLbKskIBpGuiANjV,AnnenMayKantereit,191506,62,0.756,0.573,3,-10.396,0,0.0399,0.15400,0.003500,0.2210,0.4490,93.026
9,Bye Felicia,Studying Abroad,39RwDjX4s7OuUvlutGjBZI,Masego,158112,55,0.560,0.351,8,-9.817,0,0.1250,0.67200,0.070200,0.2410,0.6230,200.156


In [30]:
sample_n = (25-len(new_playlist_df))//2
sample_n

5

In [31]:
new_playlist_df = pd.concat([new_playlist_df,filtered_saved_m.sample(sample_n),filtered_saved_t.sample(sample_n)])
new_playlist_df.reset_index(drop = True, inplace= True)

In [32]:
new_playlist_df

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Who - Single Version,Who,0CjBORMsmiQNe3vPDcNIvk,Modeselektor,207988,43,0.760,0.910,7,-8.472,1,0.0484,0.044200,0.006620,0.1080,0.39200,142.982
1,Bläulich,Treppenhaus,2WRTnY0slmFgWcrmEr8dPj,Apache 207,196213,70,0.790,0.704,10,-7.935,0,0.4170,0.069000,0.000658,0.1130,0.21200,154.007
2,The Beautiful & Damned,The Beautiful & Damned,2WWruw7ul9N7eqoHELyMc2,G-Eazy,189306,61,0.656,0.804,8,-5.191,0,0.3630,0.173000,0.000000,0.8370,0.31400,125.882
3,ABER WO BIST DU,NACHT,2dGtLJysTSI9cbQ6TulL8V,ELIF,185274,55,0.780,0.592,9,-6.020,0,0.0631,0.230000,0.000000,0.1840,0.70500,148.057
4,FREUNDE,NACHT,0VlFGxW4bynvu8j71kK6ai,ELIF,177740,56,0.786,0.705,3,-5.419,0,0.0852,0.330000,0.000000,0.1260,0.46900,82.019
5,I Mean It (feat. Remo),These Things Happen,6jmTHeoWvBaSrwWttr8Xvu,G-Eazy,236480,73,0.712,0.562,10,-6.008,1,0.1290,0.125000,0.000000,0.1360,0.14200,140.000
6,ALASKA,NACHT,2YnYp5f38UP6fvf7q2FnPm,ELIF,217766,48,0.743,0.671,1,-5.594,0,0.0589,0.058100,0.000000,0.1950,0.48400,76.998
7,FEUER,NACHT,0Se4w42WIJiTgld4SYbv8S,ELIF,203386,45,0.769,0.604,8,-6.769,1,0.0455,0.039000,0.000277,0.0738,0.22700,97.996
8,Lambo Lambo,KitschKrieg,7oqvRZNv4dUV8CgQWtIAMe,KitschKrieg,214991,59,0.876,0.400,5,-9.748,0,0.1290,0.494000,0.000009,0.1000,0.48400,144.938
9,Wealth,Who Else,5aOlYhQsp75cgPov4yjWIe,Modeselektor,247218,42,0.765,0.452,1,-12.346,1,0.0856,0.020400,0.000057,0.0727,0.33100,137.972


In [33]:
seed_tracks = new_playlist_df["track_id"].tolist()
#seed_artists = artists_m["name"].tolist() + artists_t["name"].tolist()

In [36]:
len(seed_tracks)

25

In [34]:
seed_tracks[:5], seed_tracks[5:10], seed_tracks[10:15]

(['0CjBORMsmiQNe3vPDcNIvk',
  '2WRTnY0slmFgWcrmEr8dPj',
  '2WWruw7ul9N7eqoHELyMc2',
  '2dGtLJysTSI9cbQ6TulL8V',
  '0VlFGxW4bynvu8j71kK6ai'],
 ['6jmTHeoWvBaSrwWttr8Xvu',
  '2YnYp5f38UP6fvf7q2FnPm',
  '0Se4w42WIJiTgld4SYbv8S',
  '7oqvRZNv4dUV8CgQWtIAMe',
  '5aOlYhQsp75cgPov4yjWIe'],
 ['26Td0OU1dTm3Je3ynSLC1a',
  '5bJ1DrEM4hNCafcDd1oxHx',
  '72aW82jpOZ6L0Ran6cZK2q',
  '0lwUE9DGjdbbuwbPbxZyzw',
  '5FVd6KXrgO9B3JPmC8OPst'])

In [38]:
recomms = sp_m.recommendations(seed_tracks = seed_tracks[:5],limit = 25)

In [47]:
recomms["tracks"][0]["name"]

'Unforgettable'

In [51]:
recomms["tracks"][0]["album"]["name"]

'Jungle Rules'

In [54]:
recomms["tracks"][0]["album"]["artist"]

TypeError: list indices must be integers or slices, not str

In [56]:
recomms["tracks"][0]["id"] #track_id

'3B54sVLJ402zGa6Xm4YGNe'

In [57]:
recomms["tracks"][0]["name"] #track_name

'Unforgettable'

In [59]:
recomms["tracks"][0]["album"]["name"] #album
#track_name	album	track_id	artist	duration	popularity

'Jungle Rules'

In [68]:
recomms["tracks"][0]["artists"][0]["name"] #artist

'French Montana'

In [72]:
recomms["tracks"][0]["duration_ms"] #duration

233901

In [73]:
recomms["tracks"][0]["popularity"] #popularity

80

In [74]:
def create_df_recommendations(api_results):
    track_name = []
    track_id = []
    artist = []
    album = []
    duration = []
    popularity = []
    for items in api_results['tracks']:
        try:
            track_name.append(items['name'])
            track_id.append(items['id'])
            artist.append(items["artists"][0]["name"])
            duration.append(items["duration_ms"])
            album.append(items["album"]["name"])
            popularity.append(items["popularity"])
        except TypeError:
            pass
        df = pd.DataFrame({ "track_name": track_name, 
                                "album": album, 
                                "track_id": track_id,
                                "artist": artist, 
                                "duration": duration, 
                                "popularity": popularity})

    return df

In [77]:
append_audio_features(create_df_recommendations(recomms),sp_m)

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Unforgettable,Jungle Rules,3B54sVLJ402zGa6Xm4YGNe,French Montana,233901,80,0.726,0.7690,6,-5.043,1,0.1230,0.02930,0.010100,0.1040,0.733,97.985
1,GEH DEIN WEG (feat. Loredana),Maximum III+,7y35GCh5HbhdRegcKybdeR,KC Rebell,184993,45,0.802,0.7270,5,-5.480,1,0.1210,0.01480,0.000000,0.1130,0.581,148.018
2,Lissabon,Lissabon,55LeWPQonRaIdZLaFMwwdC,Philipp Dittberner,136271,64,0.971,0.7620,0,-3.080,1,0.1560,0.18300,0.000002,0.0688,0.851,118.013
3,Bout You,Indxgo,53TdRJ6EuIrNW2DROxzp7t,Xuitcasecity,184993,43,0.790,0.8440,1,-3.467,1,0.1060,0.23100,0.000000,0.3750,0.705,112.513
4,Defender,Murmurations,7m0oNdXFP2n9V3fpgHyQ9T,Simian Mobile Disco,444853,37,0.608,0.7270,5,-10.313,0,0.0477,0.00465,0.156000,0.0806,0.516,132.046
5,Mary,Fieber,2Jij6ebp6ToNKOn5DWlGxV,Bausa,158282,60,0.652,0.7020,4,-8.124,1,0.0803,0.03000,0.000000,0.0831,0.820,151.995
6,"Für immer - aus Sing meinen Song, Vol. 7",Treppenhaus (Deluxe),6Hq4P0LtLjFeq7CDxjFf4g,LEA,204706,42,0.409,0.3650,5,-9.163,1,0.0346,0.71600,0.000000,0.4520,0.231,122.136
7,Roter Sand,Roter Sand,6MyqEEcJsXMzyu5BCDz53H,Emilio,144760,54,0.475,0.7080,3,-5.395,1,0.2700,0.16200,0.000000,0.1790,0.458,130.060
8,Endorphine,Endorphine,3rxFFtIbmz4QXZGGEEsuSj,TheDoDo,172596,68,0.686,0.5670,10,-6.249,0,0.0889,0.00396,0.001680,0.3100,0.204,181.889
9,Elbe,Elbe,2kvJnNMqnimkCZUUWjOINy,Bozza,195653,76,0.736,0.5590,5,-6.738,0,0.0605,0.10800,0.000019,0.1450,0.131,94.996


In [82]:
recomm_dfs = []
for i in range(5,26,5):
    recomms = sp_m.recommendations(seed_tracks = seed_tracks[i-5:i],limit = 25)
    recomms_df = append_audio_features(create_df_recommendations(recomms),sp_m)
    recomm_dfs.append(recomms_df)
recomms_df = pd.concat(recomm_dfs)

In [84]:
recomms_df.reset_index(drop = True, inplace= True)

In [85]:
recomms_df

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Unbeschreiblich,Unbeschreiblich,2EiA3UYmA615vJLD0Kq7Bh,Bengio,198026,44,0.612,0.842,11,-6.017,0,0.1040,0.0245,0.000077,0.0625,0.5000,80.042
1,Gib mir dein Lächeln,Alles was ich hab,4MiiK4wffgx8yF3IMmcIid,VONA,209753,51,0.703,0.564,7,-8.795,1,0.2530,0.3170,0.000000,0.1540,0.4850,173.886
2,Loreley,1977,5plDcIUqvmCsbc1rvJRfk1,Kölsch,345000,54,0.781,0.509,9,-7.793,0,0.1220,0.2910,0.000022,0.1120,0.0311,127.964
3,In meiner Erinnerung,Schritte,7hEogFloi0A60JMFfgVi3l,Silbermond,225453,55,0.590,0.464,1,-9.988,1,0.0433,0.7490,0.000055,0.1470,0.2560,124.806
4,That Far,That Far,1BjiBbVzBMREvXiT6X5tHi,6LACK,185865,54,0.746,0.607,10,-4.710,0,0.1870,0.2390,0.000019,0.0929,0.2470,83.828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,"WDYW (feat. Lil Uzi Vert, A$AP Ferg & Rich The...",Papi Gordo,4afMG2PlvFfX976bZJvHmb,Carnage,233000,39,0.720,0.816,5,-3.313,1,0.1430,0.2070,0.000000,0.8130,0.4870,125.074
121,Bayuda,Midnight Voyage,6pRT0rjiJCtFLE5rdu9HX0,Tunnelvisions,337250,33,0.709,0.688,7,-7.977,1,0.0362,0.0733,0.832000,0.0790,0.3200,110.004
122,Сансара,Сансара,2pSfzuZd81zBaOA2LOKbdX,Basta,362718,55,0.330,0.445,4,-11.287,0,0.0404,0.3950,0.042900,0.0800,0.0655,71.664
123,Dreams and Nightmares,Dreams and Nightmares (Deluxe Edition),42zd6DYQ4o4SECmTITrM1U,Meek Mill,230693,72,0.631,0.774,11,-2.459,1,0.3000,0.4680,0.000000,0.2810,0.4400,79.745
